In [ ]:
!pip install transformers==3.1.0

     |████████████████████████████████| 884 kB 2.7 MB/s 
     |████████████████████████████████| 895 kB 57.8 MB/s 
     |████████████████████████████████| 1.2 MB 62.4 MB/s 
     |████████████████████████████████| 3.0 MB 51.4 MB/s 


In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("zero-shot-classification", model='valhalla/distilbart-mnli-12-3',device=0)

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [ ]:
def grater_than(n):
    for i in range(len(n)):
        #print(n[i])
        if float(n[i]) >= 0.60:
            n[i] = 1
        else:
            n[i] = 0
    return n
def not_zero(n):
    if n == 0:
        return 1
    else: return 0

def dic2df(output):
    df = pd.DataFrame()
    df = df.append(output)
    #df = df.drop(columns=['sequence'])
    df.score = df.scores.apply(grater_than)
    
    emo_df = pd.DataFrame()
    for i in range(df.shape[0]):
        emo_d = {}
        for j in range(7):
            emo_d[df.labels[i][j]] = df.scores[i][j]
            emo_d['sequence'] = df.sequence[i]
        emo_df = emo_df.append(emo_d,ignore_index=True)

    emo_df['neutral'] = emo_df.sum(axis=1)
    emo_df.neutral = emo_df.neutral.apply(not_zero) 
    return emo_df


In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        #self.tokenizer = tokenizer
        #self.max_len = max_len
        
    def __getitem__(self, index):
        text = str(self.data.sw_Text[index])
        text = " ".join(text.split())
        #target = torch.tensor(self.data.Emotions[index].astype(int))
        return text

    
    def __len__(self):
        return self.len

In [ ]:
path = '/content/drive/MyDrive/NLP/Refined_Scala_test_data.csv'
scala_dataset = pd.read_csv(path)

In [ ]:
scala_dataset.head()

,Unnamed: 0,Content_Id,Article_Title_Txt,Locale_Cd,comment_ts,Text,sw_Text
0,0,HT204168,NaN,en_US,17-01-18 20:19,I cant accept FaceTime calls,i can t accept facetime calls
1,1,HT204053,NaN,en_US,29-07-17 16:19,genius bar appointment,genius bar appointment
2,2,HT207043,NaN,en_US,18-01-18 22:07,So basically if my phone gets wet there isnt ...,so basically if my phone gets wet there isn t ...
3,3,HT204385,NaN,en_US,12-02-18 21:10,Why do you force people to deauthorize all com...,why do you force people to deauthorize all com...
4,4,BK_BF0D7D2D2C2DCA6D2A6301E487ABFD7E_AMSA9337677E,NaN,en_US,05-02-18 21:14,my device is listed in the audio device list ...,my device is listed in the audio device list a...


In [ ]:

infrance_dataset = MyDataset(scala_dataset)

In [ ]:
VALID_IN_SIZE = 8
infrance_params = {'batch_size': VALID_IN_SIZE,
                'shuffle': False,
                'num_workers': 0
                }
infrance_loader = DataLoader(infrance_dataset, **infrance_params)

In [ ]:
candidate_labels = ['worry',
    'sad',
    'frustrate',
    'anger',
    'surprise',
    'love',
    'happy']
big_df = pd.DataFrame()
for _, data in enumerate(infrance_loader, 0):
    text = data
    output = classifier(text, candidate_labels,
                    hypothesis_template='The sentiment of this review is {}'
                    ,multi_class=True)
    df = dic2df(output)
    big_df = pd.concat([big_df,df])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [ ]:
big_df = big_df.reset_index(drop=True)

In [ ]:
big_df.to_csv('/content/drive/MyDrive/NLP/inference_of_test_data.csv')

In [ ]:
big_df.tail()

,anger,frustrate,happy,love,sad,sequence,surprise,worry,neutral,emotions
209721,0.0,0.0,1.0,0.0,0.0,need an appointment with the genius tomorrow a...,0.0,0.0,0,
209722,0.0,0.0,0.0,0.0,0.0,need more detail on ssl,0.0,0.0,1,
209723,0.0,0.0,0.0,0.0,0.0,tried to update ios. got the connect to itunes...,0.0,0.0,1,
209724,0.0,0.0,0.0,0.0,0.0,how to upgrade ios 7.1.2 on iphone 4 to ios 9,0.0,0.0,1,
209725,0.0,0.0,0.0,0.0,0.0,doesn t say why it happened,0.0,0.0,1,


In [ ]:
scala_dataset.tail()

In [ ]:
labels = ['neutral','worry',
    'sad',
    'frustrate',
    'anger',
    'surprise',
    'love',
    'happy']
big_df["emotions"] = ' '
def return_name(n,name):
    #print(n)
    if n ==0:
        return ''
    else:
        return name

for emo in labels:
    a = 0
    for i in range(big_df.shape[0]):
        
        if i%50000 == 0:
            a = a+1
            print(f'step {a} of emo {emo}')
        big_df["emotions"][i] =  big_df["emotions"][i] +' '+ return_name(big_df[emo][i],emo)

step 1 of emo neutral


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


step 2 of emo neutral
step 3 of emo neutral
step 4 of emo neutral
step 5 of emo neutral
step 1 of emo worry
step 2 of emo worry
step 3 of emo worry
step 4 of emo worry
step 5 of emo worry
step 1 of emo sad
step 2 of emo sad
step 3 of emo sad
step 4 of emo sad
step 5 of emo sad
step 1 of emo frustrate
step 2 of emo frustrate
step 3 of emo frustrate
step 4 of emo frustrate
step 5 of emo frustrate
step 1 of emo anger
step 2 of emo anger
step 3 of emo anger
step 4 of emo anger
step 5 of emo anger
step 1 of emo surprise
step 2 of emo surprise
step 3 of emo surprise
step 4 of emo surprise
step 5 of emo surprise
step 1 of emo love
step 2 of emo love
step 3 of emo love
step 4 of emo love
step 5 of emo love
step 1 of emo happy
step 2 of emo happy
step 3 of emo happy
step 4 of emo happy
step 5 of emo happy


In [ ]:
big_df.emotions = big_df.emotions.apply(lambda x: x.split())

In [ ]:
big_df.tail()

,anger,frustrate,happy,love,sad,sequence,surprise,worry,neutral,emotions
209721,0.0,0.0,1.0,0.0,0.0,need an appointment with the genius tomorrow a...,0.0,0.0,0,[happy]
209722,0.0,0.0,0.0,0.0,0.0,need more detail on ssl,0.0,0.0,1,[neutral]
209723,0.0,0.0,0.0,0.0,0.0,tried to update ios. got the connect to itunes...,0.0,0.0,1,[neutral]
209724,0.0,0.0,0.0,0.0,0.0,how to upgrade ios 7.1.2 on iphone 4 to ios 9,0.0,0.0,1,[neutral]
209725,0.0,0.0,0.0,0.0,0.0,doesn t say why it happened,0.0,0.0,1,[neutral]


In [ ]:
scala_dataset.head()

,Unnamed: 0,Content_Id,Article_Title_Txt,Locale_Cd,comment_ts,Text,sw_Text
0,0,HT204168,NaN,en_US,17-01-18 20:19,I cant accept FaceTime calls,i can t accept facetime calls
1,1,HT204053,NaN,en_US,29-07-17 16:19,genius bar appointment,genius bar appointment
2,2,HT207043,NaN,en_US,18-01-18 22:07,So basically if my phone gets wet there isnt ...,so basically if my phone gets wet there isn t ...
3,3,HT204385,NaN,en_US,12-02-18 21:10,Why do you force people to deauthorize all com...,why do you force people to deauthorize all com...
4,4,BK_BF0D7D2D2C2DCA6D2A6301E487ABFD7E_AMSA9337677E,NaN,en_US,05-02-18 21:14,my device is listed in the audio device list ...,my device is listed in the audio device list a...


In [ ]:
ans1 = scala_dataset[['Content_Id','Text']]
ans2 = big_df[['emotions']]
ans = pd.concat([ans1,ans2],axis=1)

In [ ]:
ans.head()

,Content_Id,Text,emotions
0,HT204168,I cant accept FaceTime calls,[neutral]
1,HT204053,genius bar appointment,[neutral]
2,HT207043,So basically if my phone gets wet there isnt ...,[sad]
3,HT204385,Why do you force people to deauthorize all com...,"[worry, sad, frustrate, anger, surprise]"
4,BK_BF0D7D2D2C2DCA6D2A6301E487ABFD7E_AMSA9337677E,my device is listed in the audio device list ...,[neutral]


In [ ]:
ans.to_csv('/content/drive/MyDrive/NLP/Scala_summbition.csv')